In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [3]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

# 1 data&feat

In [6]:
max_features = 20000

In [7]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)

/Users/zhangjiaqi/workingspace/anaconda3/envs/ml3/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/zhangjiaqi/workingspace/anaconda3/envs/ml3/lib/python3.7/site-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [8]:
from keras.preprocessing.sequence import pad_sequences

In [12]:
maxlen=200

In [14]:
x_train = pad_sequences(x_train,maxlen=maxlen,padding='post',truncating='post')
x_test = pad_sequences(x_val,maxlen=maxlen,padding='post',truncating='post')

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10*128).cache().batch(128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
eval_ds = tf.data.Dataset.from_tensor_slices((x_test,y_val)).shuffle(10*128).cache().batch(128)

# 2 model

In [22]:
#from tf.keras.models import Model
#from tf.keras.layers import Input,Dense,Embedding,LSTM,Bidirectional

In [23]:
input_x = tf.keras.layers.Input(shape=(maxlen,),name='input_x')

In [24]:
embed_x = tf.keras.layers.Embedding(input_dim=max_features,output_dim=64)(input_x)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
#embed_x: batch_size,maxlen,64
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=32,return_sequences=True))(embed_x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=32))(x)

In [28]:
x = tf.keras.layers.Dense(units=32,activation='relu')(x)
out = tf.keras.layers.Dense(units=1,activation='sigmoid')(x)

In [29]:
nn = tf.keras.models.Model(input_x,out)

In [30]:
nn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 64)           1280000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 64)           24832     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,331,777
Trainable params: 1,331,777
Non-trainable params: 0
___________________________________________________

In [31]:
nn.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,
           metrics=[tf.keras.metrics.binary_accuracy,tf.keras.metrics.binary_crossentropy])

In [32]:
nn.fit(train_ds,validation_data=eval_ds,epochs=2)

Train on 196 steps, validate on 196 steps
Epoch 1/2
196/196 [==============================] - 1665s 8s/step - loss: 0.4844 - binary_accuracy: 0.7489 - binary_crossentropy: 0.4844 - val_loss: 0.3668 - val_binary_accuracy: 0.8387 - val_binary_crossentropy: 0.3670
Epoch 2/2
196/196 [==============================] - 1626s 8s/step - loss: 0.2372 - binary_accuracy: 0.9100 - binary_crossentropy: 0.2377 - val_loss: 0.4297 - val_binary_accuracy: 0.8432 - val_binary_crossentropy: 0.4296
